In [2]:
import pandas as pd
import numpy as np
import json
from datasets import Dataset, Value, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import average_precision_score, precision_score, recall_score, f1_score
from fuzzywuzzy import fuzz
from tqdm import tqdm
import torch

2025-05-08 23:25:15.432505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746746715.651352      97 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746746715.712435      97 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
device

'cuda'

In [ ]:
train_data = []
with open("data/training-data.ndjson", "r", encoding="utf-8") as f:
    train_data = [json.loads(line) for line in f if line.strip()]  # skip blank lines
train_df = pd.DataFrame(train_data)
train_df = train_df[['content', 'keywords']]

In [ ]:
test_data = []
with open("data/test-data.ndjson", "r", encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f if line.strip()]  # skip blank lines
test_df = pd.DataFrame(test_data)
test_df = test_df[['content', 'keywords']]

In [8]:
train_df['content'] = train_df['content'].str.replace("\n\n", " ", regex=False)
train_df['content'] = train_df['content'].str.replace("\n", " ", regex=False)
train_df['content'] = train_df['content'].str.strip()

In [9]:
test_df['content'] = test_df['content'].str.replace("\n\n", " ", regex=False)
test_df['content'] = test_df['content'].str.replace("\n", " ", regex=False)
test_df['content'] = test_df['content'].str.strip()

In [10]:
train_df.iloc[0, :]['keywords']

['Video saliency prediction', 'feature alignment', 'deformable']

In [11]:
multi_label_binarizer = MultiLabelBinarizer()
multi_label_binarizer.fit(train_df['keywords'].tolist() + test_df['keywords'].tolist())

MultiLabelBinarizer()

In [12]:
train_df['labels'] = multi_label_binarizer.transform(train_df['keywords']).tolist()
test_df['labels'] = multi_label_binarizer.transform(test_df['keywords']).tolist()

In [13]:
train_dataset = Dataset.from_pandas(train_df[['content', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['content', 'labels']])

In [14]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
def tokenize_inputs(entry):
    return tokenizer(entry['content'], padding="max_length", truncation=True, max_length=512)

In [16]:
train_tokenized = train_dataset.map(tokenize_inputs, batched=True)
test_tokenized = test_dataset.map(tokenize_inputs, batched=True)

train_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_tokenized = train_tokenized.cast_column("labels", Sequence(Value("float32")))
test_tokenized = test_tokenized.cast_column("labels", Sequence(Value("float32")))

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased',
                                           num_labels=len(multi_label_binarizer.classes_),
                                           problem_type="multi_label_classification")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
                    output_dir="test_output_dir",
                    #run_name="test-run",
                    eval_strategy='epoch',
                    logging_strategy='epoch',
                    save_strategy='epoch',
                    learning_rate=5e-4,
                    per_device_train_batch_size=8,
                    per_device_eval_batch_size=4,
                    num_train_epochs=5,
                    weight_decay=1e-4,
                    load_best_model_at_end=True,
                    metric_for_best_model="eval_loss",
                    gradient_accumulation_steps=2
                    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_tokenized,
            eval_dataset=test_tokenized,
            tokenizer=tokenizer,
)

/tmp/ipykernel_97/99076126.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.011200,0.001441
2,0.001300,0.001409
3,0.001200,0.001424
4,0.001200,0.001443
5,0.001200,0.001450


TrainOutput(global_step=2500, training_loss=0.003228331685066223, metrics={'train_runtime': 3396.7033, 'train_samples_per_second': 11.776, 'train_steps_per_second': 0.736, 'total_flos': 1.404852830208e+16, 'train_loss': 0.003228331685066223, 'epoch': 5.0})

In [21]:
def is_fuzzy_match(pred, true_keywords):
    for true_kw in true_keywords:
        if pred in true_kw or true_kw in pred:
            return True
    return False

In [ ]:
# Get model predictions on test set
predictions = trainer.predict(test_tokenized)
logits = predictions.predictions
true_labels = predictions.label_ids

# Apply sigmoid to get probabilities
probs = 1 / (1 + np.exp(-logits))

# Get top-5 predicted label indices
top5_preds = np.argsort(probs, axis=1)[:, -5:][:, ::-1]


inv_mlb = {i: kw for i, kw in enumerate(multi_label_binarizer.classes_)}
predicted_keywords = [[inv_mlb[idx] for idx in row] for row in top5_preds]
true_keywords = [[inv_mlb[i] for i in np.where(row == 1)[0]] for row in true_labels]


tp, fp, fn = 0, 0, 0
ap5_scores = []

for preds, truths in zip(predicted_keywords, true_keywords):
    matched_preds = 0
    matched_truths = set()

    for pred in preds:
        if is_fuzzy_match(pred, truths):
            matched_preds += 1

    tp += matched_preds
    fp += len(preds) - matched_preds
    fn += len(truths) - matched_preds
    ap5_scores.append(matched_preds / 5)


precision = tp / (tp + fp + 1e-8)
recall = tp / (tp + fn + 1e-8)
f1 = 2 * precision * recall / (precision + recall + 1e-8)
map5 = np.mean(ap5_scores)


print(f"Precision (Top-5, Fuzzy): {precision:.4f}")
print(f"Recall (Top-5, Fuzzy):    {recall:.4f}")
print(f"F1 Score (Top-5, Fuzzy):  {f1:.4f}")
print(f"MAP@5 (Fuzzy):            {map5:.4f}")

Precision (Top-5, Fuzzy): 0.0762
Recall (Top-5, Fuzzy):    0.0710
F1 Score (Top-5, Fuzzy):  0.0735
MAP@5 (Fuzzy):            0.0762
